- fdisk(MBR)
    - fdisk -l 查看系统磁盘和分区详情
    - fdisk 磁盘
        - m 显示帮助信息
        - n 新建一个分区
        - p 查看分区
        - d 删除一个分区
        - q 退出不保存
        - w 退出保存
        - t 改变分区类型
        - l 查看分区类型
    - 步骤
        1. 利用fdisk工具分区
            - 让分区立即生效
                - partprobe 分区 
        2. 进行文件系统创建
            mkfs -t  ext4|xfs|ext3 分区名称
        3. 创建挂在点，一般在mnt目录
        4. 进行文件系统挂载
            - 永久挂载 vim /etc/fstab ，此文件为开机自动引导文件系统
            - /dev/mapper/centos-swap swap                    swap    defaults        0 0
                - 挂载的文件系统：/dev/mapper/centos-swap
                - 挂载点：swap
                - 文件系统类型：swap
                - 挂载选项：defaults
                - 是否需要备份 0-不需要 1-需要：0
                - 开机磁盘检查顺序（0-不检查 1-优先检查 2-其次）：0
        5. 查看挂载
            mount
            df -hT
        6. 卸载
            umount 
        7. 创建交换分区
            1. 分区
            2. 创建交换分区文件系统
                - mkswap 分区
            3. 挂载
                - swapon 分区
            4. 查看挂载
                swapon -s
                swapon -a 重新挂载

- gdisk(GPT)
    - 同fdisk

- 查看磁盘UUID
    - blkid

- LVM 逻辑卷管理
    - linux环境下对磁盘分区进行管理的一种机制
    - 建立在磁盘和分区上的一个逻辑层，来提高磁盘分区管理的灵活性
    - 注意：
        屏蔽了底层磁盘的布局，便于动态调整磁盘容量
        /boot分区不能应用LVM机制，存放引导文件
    - 基本概念
        - PV(Physical volume) 物理卷
            - 利用分区工具（fdisk）得到的普通分区，也可以是整块磁盘
            - 包含许多PE（基本单元），默认大小4M 
        - VG(Volume group) 卷组
            - 由一堆PV组成的资源组，成为卷组
        - LV(Logical volume) 逻辑卷
            - 从VG里动态划出一部分空间创建文件系统
    - LVM 创建流程
        - 创建PV
            - pvcreate 分区
        - 创建VG(Volume group) 卷组
            -  vgcreate [选项] 卷组名 PV
                - -s 指定PE大小，默认4M
        - 创建LV(Logical volume) 逻辑卷
            - lvcreate [选项] VG卷组名
                - -n 名称 逻辑卷名称
                - -L 20G  将逻辑卷直接确定为20G
                - -l 500  将逻辑卷的大小设置为500个PE块
        - 创建文件系统
            - 二选一
                -  mkfs -t ext4 /dev/mygroup/mymate
                -  mkfs -t ext4 /dev/mapper/mygroup-mymate
        - 挂载
            - 创建挂载目录
                mkdir /mnt/file
            - 一次性挂载
                mount /dev/mygroup/mymate /mnt/file
            - 永久挂载
                /dev/mygroup/mymate /mnt/file ext4  defaults 0 0
        - 查看挂载
            df -hT
    - 查看PV
        - pvdisplay 
    - 查看VG
        - vgdisplay
    - 查看LV
        - lvdisplay
    - 删除LV
        1. 卸载文件系统
            umount 挂载目录
        2. 删除逻辑卷
            lvremove 逻辑卷
        3. 删除卷组
            vgremove VG名称
        4. 删除物理卷
            pvremove PV
    - 逻辑卷扩展
        - 扩展/缩减卷组
            - 扩展卷组
                - pvcreate /dev/sdd1    #创建准备扩展的PV
                - vgextend mygroup /dev/sdd1
            - 缩减卷组
                1. 移动物理区段（VG空间足够的去情况下）
                    - pvmove /dev/sdb1
                2. 缩减卷组空间
                    - vgreduce mygroup /dev/sdb1
        - 扩展逻辑卷
            - 基于XFS文件系统的逻辑卷扩展
                1. 验证卷组是否有可用的空间
                    - vgdisplay mygroup
                        - Free  PE 字段
                2. 扩展逻辑卷
                    - lvextend -L 20G /dev/mapper/mygroup-lvname
                3. 扩展文件系统
                    - xfs_growfs /mnt/xfs_dir/
                4. 验证是否扩展成功
                    - df -hT
            - 基于ext4文件系统的扩展
                1. 验证卷组是否有可用的空间
                    - vgdisplay mygroup
                        - Free  PE 字段
                2. 扩展逻辑卷
                    - lvextend -L 20G /dev/mapper/mygroup-lvname
                3. 扩展文件系统
                    - resize2fs /dev/mapper/mygroup-tlxy 
                4. 验证是否扩展成功
                    - df -hT
            - 注意：
                - xfs_growfs 和 resize2fs 之间主要区别是为识别文件系统而传递参数
                - xfs_growfs 采用挂载点，resize2fs采用逻辑卷名称
                - lvextend 扩展参数
                    - L 10G   将逻辑卷的大小正好调整为10G
                    - L +10G  将逻辑卷的大小在原基础上增加10G
                    - l 1000  将逻辑卷的大小正好调整为1000个PE块
                    - l +1000 将逻辑卷的大小在原基础上增加1000个PE块
                    - l +50%FREE  向LV中添加VG中当前可用空间的50%

- RAID
    - 分类
        - RAID 0
            - 无差错校验RAID
            - 至少2块磁盘
            - 无数据保护
            - 读写效率最高
            - 其中一块磁盘失效，整个RAID失效
            - 利用率100%
            - 使用：图形工作站
        - RAID 1
            - 镜像结构RAID
            - 至少两块磁盘
            - 磁盘互为镜像
            - 利用率1/n
            - 可以损坏 n-1块
            - 使用：系统安装
        - RAID 3
            - 带奇偶校验的分条RAID
            - 至少3块磁盘，其中一块磁盘存放校验信息
            - 只读有效数据，写入数据进行校验
            - 最多允许环一块磁盘
            - 利用率：（n-1）/n
            - 注：当写入数据较少时，会产生“写惩罚”
        - RAID 5
            - 分布式奇偶校验
            - 至少3块磁盘
            - 最多允许环一块磁盘
            - 利用率：（n-1）/n
            - 常用
        - RAID 6
            - 最多允许环两块磁盘
        - RAID 10
            - 将将镜像（RAID1）和条带（RAUID0）进行组合的RAID，先做RAID1，在做RAID0
        - RAID 50
            - 将RAID5和RAUID0进行组合的RAID，先做RAID5，在做RAID0
    - 通过mdadm 命令实现软件RAID配置
        - 安装 mdadm
            - yum install mdadm -y
        - mdadm 常用模式：
            - Assemble：装配模式，将原来属于一个阵列的每个块设备组装为阵列
            - build： 创建一个不需要元数据的阵列（超级块），每个设备没有元数据块
            - create： 创建一个新的阵列，每个设备都有超级块
                --create == -c 创建一个新的阵列 
                --level == -l 设置RAID组级别
                --raid-devices == -n  设置设备活动个数
                --spare-devices == -x 设置备用盘的个数
                --chunk == -c chunk默认64K
            - Monitor 监控模式，监控RAID状态，可以实现全局热备
                --follow == -F  选择监控模式
                --syslog == -y  时间日志记录
                --delay == -d   设置RAID轮循，默认60s
                --test  == -t   生成警告信息
            - Grow 修改阵列属性（阵列磁盘个数，使用容量）
                --grow == -G 修改阵列大小或设备数量
                --rail-devices == -n 活动设备个数
                --chunk == -c 设置chunk大小
                --level == -l 设置RAID等级
                --name == -N  设置阵列名称
            - Incremental Assembly：添加一个设备到阵列中
            - Misc 报告或者修改阵列中相关设备信息
                --query == -Q 查询一个RAID或者RAID组件信息
                --detail == -D 查看RAID组详细信息
                --stop == -s 停止RAID组
            - Auto-detect ：要求在linux内核启动时检测阵列
        - RAID0 实现过程
            1. 安装或更新 mdadm
                - yum install mdadm -y
            2. 准备RAID成员磁盘     
                - fdisk /dev/sdb
                    - 使用‘t’改变分区类型
                - fdisk /dev/sdc同上
            3. 准备完磁盘后，检查磁盘是否正确定义RAID
                - mdadm --examine /dev/sd{b,c}
                - mdadm --examine /dev/sd{b,c}1
            4. 创建RAID0(/dev/md0)
                mdadm -C /dev/md0 -l raid0 -n 2 /dev/sd{b,c}1 
            5. 查看RAID组状态
                cat /proc/mdstat
               查看RAID级别
                mdadm -E /dev/sd[b,c]1 
               查看RAID设备
                mdadm --detail /dev/md0
            6. 创建文件系统
                mkfs -t ext4 /dev/md0
            7. 创建挂载目录并进行永久挂载
                - mkdir /mnt/raid0
                - vim /etc/fstab
                    /dev/md0 /mnt/raid0/    ext4    defaults 0 0 
            8. 保存配置
                mdadm -E -s -v >> etc/mdadm.conf
            9. 使用RAID
                echo "这是一个RAID0阵列测试文件" > /mnt/raid0/test.txt